In [508]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize 
from pandas import read_csv
import logging
from functools import reduce
import csv
from collections import defaultdict
import os
import uuid

In [592]:
#etl = 'kobo2elastic'
#etl = 'curis2elastic'
#etl = 'oldcuris2newcuris'
etl = 'cuartero2newaqm'
#etl = 'cambodia2newaqm'

input_schema_file = ''
input_data_file = ''
mapping_file = ''

root_object = 'resident'

if etl == 'curis2elastic':
    #old curis to elasticsearch
    input_schema_file = 'schema/input/curisSchema.1-item.json'
    input_data_file = 'data/curisData.1-items.json'
    mapping_file = 'schema/map/couchbase2elastic.map.csv'
    output_dir = 'file/curisSchema/'
    
elif etl == 'kobo2elastic':
    #kobo to elasticsearch
    input_schema_file = 'schema/input/koboSchema.1-item.json'
    input_data_file = 'data/koboData.2-items.json'
    mapping_file = 'schema/map/kobo2elastic.map.csv'
    output_dir = 'file/koboSchema/'
    
elif etl == 'oldcuris2newcuris':
    #kobo to elasticsearch
    input_schema_file = 'schema/input/curisSchema.1-item.json'
    input_data_file = 'data/curisData.1-item.json'
    mapping_file = 'schema/map/kobo2elastic.map.csv'
    output_dir = 'file/curisSchema/'
    
elif etl == 'cambodia2newaqm':
    #kobo to elasticsearch
    input_schema_file = 'schema/input/KHMSchema.json'
    input_data_file = 'data/KHM.3-items.json'
    mapping_file = 'schema/map/kobo2elastic.map.csv'
    output_dir = 'file/KHMSchema/'

elif etl == 'cuartero2newaqm':
    input_schema_file = 'schema/input/curisSchema.1-item.json'
    #input_data_file = 'data/Isabela.2-items.json'
    #mapping_file = 'schema/map/Philippines/2.2.AQMHealthInfoQuestionsV1.csv'
    #mapping_file = 'schema/map/Philippines/2.2.AQMHealthInfoQuestionsV1.csv'
    mapping_file = 'schema/map/Philippines/2.1.AQMGeneralQuestionsV1.csv'
    datelog_dir = 'couchbase-curis-2019-06-21-cuartero'
    processed_dir    = 'data/processed/' + datelog_dir + '/'
    tmp_dir    = 'data/processed/' + datelog_dir + '/tmp/'
    output_dir = 'data/processed/' + datelog_dir + '/'
    merged_dir = 'data/merged/' + datelog_dir + '/'
    


## TODO: CREATE FOLDER DIRECTORY MKDIR
## TODO: PARSED FIELD TYPE: LIST -- merged into single object

## TODO: FIX ID MERGING

## TODO: FIX _id into profileId

# READ MAPPING FILE

In [604]:
#def _get_mapping_fields():
_mapping_df = pd.DataFrame()
_mapping_df = read_csv(mapping_file).sort_values(['source_key']).replace(np.nan,'',regex=True)
_mapping_df

,source_key,source_type,destination_key,destination_key.1,data_type,data_source,data_format,default_value,lookup_value
0,profiles.civil_status,string,answers.civilStatus,string,existing,original,,,
2,profiles.education,string,answers.highestEducation,string,existing,original,,,
4,profiles.employment.is_employed,string,answers.employmentStatus,string,existing,original,,,
3,profiles.employment.nature,string,answers.employmentArea,string,existing,original,,,
1,profiles.religion,string,answers.religion,string,existing,original,,,
5,,,dateUpdated,timestamp,new,generated,timestamp,,
6,,,dateCreated,timestamp,new,generated,timestamp,2019-06-27T20:36:43.993+08:00,
7,,,formName,string,new,generated,,2.1 - AQM General Questions V1,
8,,,formId,string,new,generated,,AWviKwKDKxqtHWZMp2v4,
9,,,type,string,new,generated,,profile-related-form,


# GET REQUIRED FIELDS IN THE MAPPING

In [558]:
mapping_fields_list = []
mapping_fields_list = list(_mapping_df['source_key'].unique())
mapping_fields_list

['profiles.civil_status',
 'profiles.education',
 'profiles.employment.is_employed',
 'profiles.employment.nature',
 'profiles.religion',
 nan]

# GET FIELD NAMES OF THE SCHEMA using _META

In [578]:
meta_headers_df = pd.DataFrame()
meta_headers_df = pd.read_csv( 'data/processed/couchbase-curis-2019-06-21/_meta.csv')
meta_headers_df

,file_name,field_name,field_type
0,resident,middle_name,primitive
1,resident,birthdate,primitive
2,resident,id,primitive
3,resident,type,primitive
4,resident,first_name,primitive
5,resident,last_name,primitive
6,resident,last_name_suffix,primitive
7,resident,organization,primitive
8,resident,nhid,primitive
9,resident,gender,primitive


In [579]:
match_headers_df = pd.DataFrame()
match_headers_df = meta_headers_df[meta_headers_df['field_name'].isin(mapping_fields_list)]
match_headers_df = match_headers_df.sort_values(['file_name','field_name']).reset_index(drop=True)
match_headers_df

,file_name,field_name,field_type
0,profiles,profiles.civil_status,primitive
1,profiles,profiles.education,primitive
2,profiles,profiles.employment.is_employed,primitive
3,profiles,profiles.employment.nature,primitive
4,profiles,profiles.religion,primitive


# CREATE DEFAULT DICT FOR FILENAME AS KEY and FIELD NAMES AS VALUE

In [580]:
filename_per_field_dd = defaultdict(list)

for index,row in match_headers_df.iterrows():
    filename = row['file_name']
    fields = row['field_name']
    filename_per_field_dd[filename].append(fields)  
    
filename_per_field_dd

defaultdict(list,
            {'profiles': ['profiles.civil_status',
              'profiles.education',
              'profiles.employment.is_employed',
              'profiles.employment.nature',
              'profiles.religion']})

## CREATE A SINGLE FILE WITH ALL HEADERS

## TODO: FIlename must be dynamic

In [563]:
file_name = 'general_questions'
file_type = '.csv'

mandatory_headers_list = ['_id','_index_map']
_column_list = mandatory_headers_list + mapping_fields_list 
empty_df = pd.DataFrame(columns = _column_list )
empty_df.to_csv(merged_dir + file_name + file_type, encoding='utf-8', mode='w',header=True,index=False)

## GET THE DATA FOR EACH HEADERS

# TODO: WHAT IS APPEND FOR DATAFRAMES JOIN BY _INDEX_ID

## MERGE JOIN BY _id and _index_map

## WRITE EMPTY CSV FOR MERGE.csv

In [597]:
test_filename_per_field_dd = {
    'health_informations': ['health_informations.allergies',
              'health_informations.blood_pressure.first_reading.diastole',
              'health_informations.blood_pressure.first_reading.systole',
              'health_informations.blood_pressure.second_reading.diastole',
              'health_informations.blood_pressure.second_reading.systole',
              'health_informations.blood_sign',
              'health_informations.blood_type',
              'health_informations.exercise_in_a_week',
              'health_informations.smoking_habit'],
    'health_informations.family_history': ['health_informations.family_history']}

test_filename_per_field_dd = {'profiles': ['profiles.civil_status','profiles.employment.is_employed','profiles.education','profiles.employment.nature','profiles.religion']}

fields_list = list(test_filename_per_field_dd.values())
flat_fields_list = [item for sublist in fields_list for item in sublist]
all_fields_list  = ['_id','_index_map'] + flat_fields_list

#os.remove(merged_dir + 'general_questions' + '.csv', )
empty_data_df = pd.DataFrame(columns=all_fields_list)
empty_data_df.to_csv(merged_dir + 'general_questions'+ '.csv', encoding='utf-8', mode='w', header=True,index=False)

In [ ]:
test_filename_per_field_dd = {
          
          'resident': ['gender','birthdate'],
          'profiles': ['profiles.employment.monthly_income'],
          'health_informations': [
              'health_informations.allergies',
              'health_informations.blood_sign',
              'health_informations.blood_type',
              'health_informations.exercise_in_a_week',
              'health_informations.smoking_habit'],
          'family_planning_and_maternal_healths': [
              'family_planning_and_maternal_healths.are_you_pregnant',
              'family_planning_and_maternal_healths.date_updated']
          }

In [281]:
test_df = pd.DataFrame()
for filename, fields in test_filename_per_field_dd.items():
    print('\n')
    print('---FILENAME: ', filename)
    print('---FIELDS: ',fields)

    
    _individual_data_df = pd.DataFrame()
    _individual_data_df = pd.read_csv(processed_dir + filename + '.csv').sort_values(['_id','_index_map'])  
    
    filename_fields_list = ['_id','_index_map'] + fields
    _individual_data_df = _individual_data_df[filename_fields_list]
    
    _required_data_df = pd.read_csv(merged_dir + 'health_informations'+ '.csv',nrows=0)
    
    for index, row in _individual_data_df.iterrows():

        #_unique_id_index_df = pd.read_csv(merged_dir + 'health_informations'+ '.csv', dtype={'_index_map': str})[['_id','_index_map']].replace(np.nan,'',regex=True)
        #_uniqued_ids_list = list(_unique_id_index_df['_id'].unique())
        #_existing_id = ''
        
        for header in row.index:
            
            data_value = row[header]
            print('HEADER :' + header + '  value: ' + str(value) )
            
            
            if header == '_id':
                if data_value in _uniqued_ids_list:
                    _existing_id = unique_id_index_df[unique_id_index_df['_id'] == data_value].index[0]
                    
                    
                    print('existing at id: ', data_value)
                    print(_existing_id)
                    _required_data_df.at['',header] = row[header]
                else:
                    print('not existing')
                    _required_data_df.at['',header] = row[header]
            else:
                _required_data_df.at['',header] = row[header]
        print('------')
        
        
        _required_data_df.to_csv(merged_dir + 'health_informations'+ '.csv', encoding='utf-8', mode='a',header=False,index=False)




---FILENAME:  resident
---FIELDS:  ['gender', 'birthdate']
HEADER :_id  value: 01/01/2009 at 08:44:13 PM GMT+08:00
existing at id:  2f920d39-319e-4bf7-90c8-6133c3337af7
0
HEADER :_index_map  value: 01/01/2009 at 08:44:13 PM GMT+08:00
HEADER :gender  value: 01/01/2009 at 08:44:13 PM GMT+08:00
HEADER :birthdate  value: 01/01/2009 at 08:44:13 PM GMT+08:00
------
HEADER :_id  value: 01/01/2009 at 08:44:13 PM GMT+08:00
existing at id:  64c2e7de-0cc8-4df5-a54f-02398652c080
1
HEADER :_index_map  value: 01/01/2009 at 08:44:13 PM GMT+08:00
HEADER :gender  value: 01/01/2009 at 08:44:13 PM GMT+08:00
HEADER :birthdate  value: 01/01/2009 at 08:44:13 PM GMT+08:00
------


---FILENAME:  profiles
---FIELDS:  ['profiles.employment.monthly_income']
HEADER :_id  value: 01/01/2009 at 08:44:13 PM GMT+08:00
existing at id:  2f920d39-319e-4bf7-90c8-6133c3337af7
0
HEADER :_index_map  value: 01/01/2009 at 08:44:13 PM GMT+08:00
HEADER :profiles.employment.monthly_income  value: 01/01/2009 at 08:44:13 PM GMT+0

In [596]:
os.remove(merged_dir + 'general_questions'+ '.csv', )

In [594]:

test_filename_per_field_dd = {
    'health_informations': ['health_informations.allergies',
              'health_informations.blood_pressure.first_reading.diastole',
              'health_informations.blood_pressure.first_reading.systole',
              'health_informations.blood_pressure.second_reading.diastole',
              'health_informations.blood_pressure.second_reading.systole',
              'health_informations.blood_sign',
              'health_informations.blood_type',
              'health_informations.exercise_in_a_week',
              'health_informations.smoking_habit'],
    'health_informations.family_history': ['health_informations.family_history']}


fields_list = list(test_filename_per_field_dd.values())
flat_fields_list = [item for sublist in fields_list for item in sublist]
all_fields_list  = ['_id','_index_map'] + flat_fields_list


empty_data_df = pd.DataFrame(columns=all_fields_list)
empty_data_df.to_csv(merged_dir + 'health_informations'+ '.csv', encoding='utf-8', mode='w', header=True,index=False)

In [350]:



for filename, fields in test_filename_per_field_dd.items():
    print('\n')
    print('---FILENAME: ', filename)
    print('---FIELDS: ',fields)

    
    _individual_data_df = pd.DataFrame()
    _individual_data_df = pd.read_csv(processed_dir + filename + '.csv').sort_values(['_id','_index_map'])  
    
    filename_fields_list = ['_id','_index_map'] + fields
    _individual_data_df = _individual_data_df[filename_fields_list]
    
    _required_data_df = pd.read_csv(merged_dir + 'health_informations'+ '.csv',nrows=0)

    for index, row in _individual_data_df.iterrows():

        #_unique_id_index_df = pd.read_csv(merged_dir + 'health_informations'+ '.csv', dtype={'_index_map': str})[['_id','_index_map']].replace(np.nan,'',regex=True)
        #_uniqued_ids_list = list(_unique_id_index_df['_id'].unique())
        #_existing_id = ''
        _index_loc = ''
        for header in row.index:
        
            _unique_id_index_df = pd.read_csv(merged_dir + 'health_informations'+ '.csv', dtype={'_index_map': str})[['_id','_index_map']].replace(np.nan,'',regex=True)
            _uniqued_ids_list = list(_unique_id_index_df['_id'].unique())
            print('UNIQUE LSIT' , _uniqued_ids_list  )
        
            value = row[header]
            _required_data_df.at['',header] = value

                
        
        _required_data_df.to_csv(merged_dir + 'health_informations'+ '.csv', encoding='utf-8', mode='a',header=False,index=False)




---FILENAME:  resident
---FIELDS:  ['gender', 'birthdate']
UNIQUE LSIT ['2f920d39-319e-4bf7-90c8-6133c3337af7', '64c2e7de-0cc8-4df5-a54f-02398652c080', '']
UNIQUE LSIT ['2f920d39-319e-4bf7-90c8-6133c3337af7', '64c2e7de-0cc8-4df5-a54f-02398652c080', '']
UNIQUE LSIT ['2f920d39-319e-4bf7-90c8-6133c3337af7', '64c2e7de-0cc8-4df5-a54f-02398652c080', '']
UNIQUE LSIT ['2f920d39-319e-4bf7-90c8-6133c3337af7', '64c2e7de-0cc8-4df5-a54f-02398652c080', '']
UNIQUE LSIT ['2f920d39-319e-4bf7-90c8-6133c3337af7', '64c2e7de-0cc8-4df5-a54f-02398652c080', '']
UNIQUE LSIT ['2f920d39-319e-4bf7-90c8-6133c3337af7', '64c2e7de-0cc8-4df5-a54f-02398652c080', '']
UNIQUE LSIT ['2f920d39-319e-4bf7-90c8-6133c3337af7', '64c2e7de-0cc8-4df5-a54f-02398652c080', '']
UNIQUE LSIT ['2f920d39-319e-4bf7-90c8-6133c3337af7', '64c2e7de-0cc8-4df5-a54f-02398652c080', '']


---FILENAME:  profiles
---FIELDS:  ['profiles.employment.monthly_income']
UNIQUE LSIT ['2f920d39-319e-4bf7-90c8-6133c3337af7', '64c2e7de-0cc8-4df5-a54f-02398652c

## TEST LIST ALLR REQUIRED FIELDS

## TEST FIELD NAME ONLY

## GET PRE_FORMED CSV

## GET DATA SOURCE -- ITERATE FILENAME HERE!!

## ITERATE GET DATA SOURCE BY EACH FILENAME

## ITERATE DATA SOURCE TO INSERT INDIVIDUALLY PER CELL ACCORDING TO HEADER

In [351]:
## MEREGE TEST

In [ ]:
test_filename_per_field_dd = {
          
          'resident': ['gender','birthdate'],
          'profiles': ['profiles.employment.monthly_income'],
          'health_informations': [
              'health_informations.allergies',
              'health_informations.blood_sign',
              'health_informations.blood_type',
              'health_informations.exercise_in_a_week',
              'health_informations.smoking_habit'],
          'family_planning_and_maternal_healths': [
              'family_planning_and_maternal_healths.are_you_pregnant',
              'family_planning_and_maternal_healths.date_updated']
          }


## FAMILY PLANNING

In [423]:
_fp_df = pd.read_csv(processed_dir + 'family_planning_and_maternal_healths' + '.csv', dtype={"_index_map": str}).sort_values(['_id','_index_map']) 
_fp_df = _fp_df[['_id','_index_map',
              'family_planning_and_maternal_healths.are_you_pregnant',
              'family_planning_and_maternal_healths.date_updated']].replace(np.nan,'',regex=True)
_fp_df 

,_id,_index_map,family_planning_and_maternal_healths.are_you_pregnant,family_planning_and_maternal_healths.date_updated
0,2f920d39-319e-4bf7-90c8-6133c3337af7,0,No,01/01/2009 at 08:44:13 PM GMT+08:00
2,2f920d39-319e-4bf7-90c8-6133c3337af7,1,Yes,03/02/2008 at 08:44:13 PM GMT+08:00
1,64c2e7de-0cc8-4df5-a54f-02398652c080,0,No,01/01/2009 at 08:44:13 PM GMT+08:00


## RESIDENT DATA

In [424]:
_resident_df = pd.read_csv(processed_dir + 'resident' + '.csv', dtype={"_index_map": str}).sort_values(['_id','_index_map']) 
_resident_df = _resident_df[['_id','_index_map','gender','birthdate']].replace(np.nan,'',regex=True)
_resident_df 

,_id,_index_map,gender,birthdate
0,2f920d39-319e-4bf7-90c8-6133c3337af7,,Male,12/23/1975
1,64c2e7de-0cc8-4df5-a54f-02398652c080,,Female,08/09/1989


## PROFILE DATA

In [598]:
_profile_df = pd.read_csv(processed_dir + 'profiles' + '.csv', dtype={"_index_map": str}).sort_values(['_id','_index_map']) 
_profile_df = _profile_df[['_id','_index_map','profiles.civil_status','profiles.employment.is_employed','profiles.education','profiles.employment.nature','profiles.religion']].replace(np.nan,'',regex=True)
_profile_df = _profile_df.replace(np.nan,'',regex=True).reset_index(drop=True)
_profile_df

,_id,_index_map,profiles.civil_status,profiles.employment.is_employed,profiles.education,profiles.employment.nature,profiles.religion
0,0003ff38-28fb-4005-9437-d276cbb9da4d,0,Single,,No Formal Education,empty,Catholic
1,0003ff38-28fb-4005-9437-d276cbb9da4d,1,Single,,No Formal Education,empty,Catholic
2,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,0,Single,,Elementary Level,empty,Catholic
3,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,1,Single,,Wala pormal na Edukasyon,empty,Catholic
4,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,2,Single,,Wala pormal na Edukasyon,empty,Catholic
5,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,3,Single,,Wala pormal na Edukasyon,empty,Catholic
6,0006de3c-57ea-46d2-89b8-e606533d3d15,0,Single,,College Level,empty,Catholic
7,0006de3c-57ea-46d2-89b8-e606533d3d15,1,Single,,College Level,empty,Catholic
8,000c23e6-6f0b-45ab-a0b5-e5d19dc023ad,0,Married,,Elementary Level,empty,Catholic
9,000cb28d-04d8-4665-90e6-fb1e2fe3c11c,0,Widowed,,Elementary Level,empty,Catholic


## HEALTH INFORATION DATA

In [548]:
_hi_df = pd.read_csv(processed_dir + 'health_informations' + '.csv', dtype={"_index_map": str}).sort_values(['_id','_index_map']) 
_hi_df  = _hi_df [['_id','_index_map',
              'health_informations.allergies',
              'health_informations.blood_pressure.first_reading.diastole',
              'health_informations.blood_pressure.first_reading.systole',
              'health_informations.blood_pressure.second_reading.diastole',
              'health_informations.blood_pressure.second_reading.systole',
              'health_informations.blood_sign',
              'health_informations.blood_type',
              'health_informations.exercise_in_a_week',
              'health_informations.smoking_habit']].replace(np.nan,'',regex=True).reset_index(drop=True)
_hi_df.head(5)

,_id,_index_map,health_informations.allergies,health_informations.blood_pressure.first_reading.diastole,health_informations.blood_pressure.first_reading.systole,health_informations.blood_pressure.second_reading.diastole,health_informations.blood_pressure.second_reading.systole,health_informations.blood_sign,health_informations.blood_type,health_informations.exercise_in_a_week,health_informations.smoking_habit
0,0003ff38-28fb-4005-9437-d276cbb9da4d,0,,,,,,empty,I don't know,0 to 1 hour,Never
1,0003ff38-28fb-4005-9437-d276cbb9da4d,1,,,,,,empty,I don't know,0 to 1 hour,Never
2,0003ff38-28fb-4005-9437-d276cbb9da4d,2,,,,,,empty,I don't know,0 to 1 hour,Never
3,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,0,,,,,,Wala ko kabalo,Wala ko kabalo,,empty
4,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,1,,,,,,Wala ko kabalo,Wala ko kabalo,,empty


## MERGE ALL DATA

## TODO: DROP '0' in index_map

In [432]:
_merge_df = _resident_df.merge(_profile_df,on=["_id","_index_map"], how="outer",  suffixes=('_x', '_y') )
_merge_df = _merge_df.merge(_hi_df,on=["_id","_index_map"], how="outer",  suffixes=('_x', '_y') )
_merge_df = _merge_df.merge(_fp_df,on=["_id","_index_map"], how="outer",  suffixes=('_x', '_y') )
_merge_df.to_csv(merged_dir + 'merge' + '.csv',index=False)
_merge_df

,_id,_index_map,gender,birthdate,profiles.employment.monthly_income,health_informations.smoking_habit,health_informations.blood_type,family_planning_and_maternal_healths.are_you_pregnant,family_planning_and_maternal_healths.date_updated
0,2f920d39-319e-4bf7-90c8-6133c3337af7,,Male,12/23/1975,NaN,NaN,NaN,NaN,NaN
1,64c2e7de-0cc8-4df5-a54f-02398652c080,,Female,08/09/1989,NaN,NaN,NaN,NaN,NaN
2,2f920d39-319e-4bf7-90c8-6133c3337af7,0,NaN,NaN,Php 13.00,Pssive,B,No,01/01/2009 at 08:44:13 PM GMT+08:00
3,64c2e7de-0cc8-4df5-a54f-02398652c080,0,NaN,NaN,Php12.00,Active,A,No,01/01/2009 at 08:44:13 PM GMT+08:00
4,2f920d39-319e-4bf7-90c8-6133c3337af7,1,NaN,NaN,NaN,Never,,Yes,03/02/2008 at 08:44:13 PM GMT+08:00


In [462]:
test_group_df = pd.read_csv(merged_dir + 'merge' + '.csv',dtype={'_index_map': str}).replace(np.nan,'',regex=True)

In [463]:
test_group_df

,_id,_index_map,gender,birthdate,profiles.employment.monthly_income,health_informations.smoking_habit,health_informations.blood_type,family_planning_and_maternal_healths.are_you_pregnant,family_planning_and_maternal_healths.date_updated
0,2f920d39-319e-4bf7-90c8-6133c3337af7,,Male,12/23/75,,,,,
1,64c2e7de-0cc8-4df5-a54f-02398652c080,,Female,8/9/89,,,,,
2,2f920d39-319e-4bf7-90c8-6133c3337af7,,,,Php 13.00,Pssive,B,No,01/01/2009 at 08:44:13 PM GMT+08:00
3,64c2e7de-0cc8-4df5-a54f-02398652c080,,,,Php12.00,Active,A,No,01/01/2009 at 08:44:13 PM GMT+08:00
4,2f920d39-319e-4bf7-90c8-6133c3337af7,1,,,,Never,,Yes,03/02/2008 at 08:44:13 PM GMT+08:00


In [459]:
_1_df = test_group_df[test_group_df['_index_map'] == '1']
_1_df

,_id,_index_map,gender,birthdate,profiles.employment.monthly_income,health_informations.smoking_habit,health_informations.blood_type,family_planning_and_maternal_healths.are_you_pregnant,family_planning_and_maternal_healths.date_updated
4,2f920d39-319e-4bf7-90c8-6133c3337af7,1,,,,Never,,Yes,03/02/2008 at 08:44:13 PM GMT+08:00


In [461]:
_0_df = test_group_df[test_group_df['_index_map'] == '']
_0_df

,_id,_index_map,gender,birthdate,profiles.employment.monthly_income,health_informations.smoking_habit,health_informations.blood_type,family_planning_and_maternal_healths.are_you_pregnant,family_planning_and_maternal_healths.date_updated
0,2f920d39-319e-4bf7-90c8-6133c3337af7,,Male,12/23/75,,,,,
1,64c2e7de-0cc8-4df5-a54f-02398652c080,,Female,8/9/89,,,,,
2,2f920d39-319e-4bf7-90c8-6133c3337af7,,,,Php 13.00,Pssive,B,No,01/01/2009 at 08:44:13 PM GMT+08:00
3,64c2e7de-0cc8-4df5-a54f-02398652c080,,,,Php12.00,Active,A,No,01/01/2009 at 08:44:13 PM GMT+08:00


## RENAME THE HEADERS USING MAPPED HEADERS

In [599]:
source_destination_keys_df = pd.DataFrame()
source_destination_keys_df = _mapping_df[['source_key','destination_key']]

In [600]:
new_column_name_dict = dict(zip(source_destination_keys_df['source_key'], source_destination_keys_df['destination_key']))
new_column_name_dict 

{'profiles.civil_status': 'answers.civilStatus',
 'profiles.education': 'answers.highestEducation',
 'profiles.employment.is_employed': 'answers.employmentStatus',
 'profiles.employment.nature': 'answers.employmentArea',
 'profiles.religion': 'answers.religion',
 nan: 'type'}

In [512]:
uuid.uuid4()

UUID('d14d5fca-5e9b-485d-ad37-b1352614fbe7')

In [601]:
#TEMPORARY: clear _index_map
_required_data_df = pd.DataFrame()
_required_data_df = _profile_df
_required_data_df.rename(columns = new_column_name_dict,inplace=True )
_required_data_df.rename(columns={'_id': 'profileId'}, inplace=True)
_required_data_df['_id'] = _required_data_df.index.to_series().map(lambda x: uuid.uuid4())
_required_data_df['_index_map'] = ''
_required_data_df

,profileId,_index_map,answers.civilStatus,answers.employmentStatus,answers.highestEducation,answers.employmentArea,answers.religion,_id
0,0003ff38-28fb-4005-9437-d276cbb9da4d,,Single,,No Formal Education,empty,Catholic,24bebdcb-c34e-412e-8a22-7a873ab7bd16
1,0003ff38-28fb-4005-9437-d276cbb9da4d,,Single,,No Formal Education,empty,Catholic,aefd6b7c-5ddc-41ec-89dd-f481507140ef
2,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,,Single,,Elementary Level,empty,Catholic,e149b937-8c25-4e35-aad0-ba875d9fbb70
3,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,,Single,,Wala pormal na Edukasyon,empty,Catholic,cd550aae-228c-4d1a-b90d-7adedeffd9b9
4,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,,Single,,Wala pormal na Edukasyon,empty,Catholic,a471503f-79f6-4f51-bd90-84d45b44df82
5,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,,Single,,Wala pormal na Edukasyon,empty,Catholic,f9d6aba0-f51f-4c64-85b8-ff0992e2e4a2
6,0006de3c-57ea-46d2-89b8-e606533d3d15,,Single,,College Level,empty,Catholic,510557fc-d08c-4455-8e3a-2006f18dfe73
7,0006de3c-57ea-46d2-89b8-e606533d3d15,,Single,,College Level,empty,Catholic,b7f46116-4155-4f18-874f-8a1bbf6e6f85
8,000c23e6-6f0b-45ab-a0b5-e5d19dc023ad,,Married,,Elementary Level,empty,Catholic,a3a0aae5-ec1c-4861-8faa-fb195a3fd716
9,000cb28d-04d8-4665-90e6-fb1e2fe3c11c,,Widowed,,Elementary Level,empty,Catholic,49af29ec-9e68-40b4-ab4d-7c79c0929d0c


## CREATE THE NEW DATA WITH DEFAULT VALUES

In [605]:
new_fields_df = _mapping_df[_mapping_df['data_type'] == 'new'][['destination_key', 'default_value']]
new_fields_df

,destination_key,default_value
5,dateUpdated,
6,dateCreated,2019-06-27T20:36:43.993+08:00
7,formName,2.1 - AQM General Questions V1
8,formId,AWviKwKDKxqtHWZMp2v4
9,type,profile-related-form


In [433]:
## TODO: Insert profileId

In [606]:
for index,row in new_fields_df.iterrows():
    _header = row['destination_key']
    _value = row['default_value']
    _required_data_df[_header] = _value


_required_data_df = _required_data_df.reset_index(drop=True)
_required_data_df.head(5)

,profileId,_index_map,answers.civilStatus,answers.employmentStatus,answers.highestEducation,answers.employmentArea,answers.religion,_id,dateUpdated,dateCreated,formName,formId,type
0,0003ff38-28fb-4005-9437-d276cbb9da4d,,Single,,No Formal Education,empty,Catholic,24bebdcb-c34e-412e-8a22-7a873ab7bd16,,2019-06-27T20:36:43.993+08:00,2.1 - AQM General Questions V1,AWviKwKDKxqtHWZMp2v4,profile-related-form
1,0003ff38-28fb-4005-9437-d276cbb9da4d,,Single,,No Formal Education,empty,Catholic,aefd6b7c-5ddc-41ec-89dd-f481507140ef,,2019-06-27T20:36:43.993+08:00,2.1 - AQM General Questions V1,AWviKwKDKxqtHWZMp2v4,profile-related-form
2,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,,Single,,Elementary Level,empty,Catholic,e149b937-8c25-4e35-aad0-ba875d9fbb70,,2019-06-27T20:36:43.993+08:00,2.1 - AQM General Questions V1,AWviKwKDKxqtHWZMp2v4,profile-related-form
3,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,,Single,,Wala pormal na Edukasyon,empty,Catholic,cd550aae-228c-4d1a-b90d-7adedeffd9b9,,2019-06-27T20:36:43.993+08:00,2.1 - AQM General Questions V1,AWviKwKDKxqtHWZMp2v4,profile-related-form
4,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,,Single,,Wala pormal na Edukasyon,empty,Catholic,a471503f-79f6-4f51-bd90-84d45b44df82,,2019-06-27T20:36:43.993+08:00,2.1 - AQM General Questions V1,AWviKwKDKxqtHWZMp2v4,profile-related-form


# WRITE TO CSV FINALIZED


In [608]:
_required_data_df.to_csv(merged_dir + 'general_questions.csv', encoding='utf-8', mode='w', header=True,index=False)

## HEALTH INFORMATIONS --ITERATION

In [387]:
data_df = pd.read_csv( 'data/processed/couchbase-curis-2019-06-21/health_informations.csv')
data_headers_list = list(data_df.columns)
data_headers_list

['_id',
 '_index_map',
 'health_informations.height',
 'health_informations.exercise_in_a_week',
 'health_informations.waist_circumference',
 'health_informations.blood_type',
 'health_informations.date_updated',
 'health_informations.vegetables_in_a_week',
 'health_informations.weight',
 'health_informations.maintenance_drugs',
 'health_informations.fruits_in_a_week',
 'health_informations.blood_sugar',
 'health_informations.high_cost_medicine',
 'health_informations.allergies',
 'health_informations.blood_sign',
 'health_informations.smoking_habit',
 'health_informations.blood_pressure.third_reading.systole',
 'health_informations.blood_pressure.first_reading.diastole',
 'health_informations.blood_pressure.first_reading.systole',
 'health_informations.blood_pressure.second_reading.diastole',
 'health_informations.blood_pressure.third_reading.diastole',
 'health_informations.blood_pressure.second_reading.systole']

In [388]:
hi_match_headers = list(set(mapping_fields_list) & set(data_headers_list))
hi_match_headers

['health_informations.exercise_in_a_week',
 'health_informations.blood_sugar',
 'health_informations.blood_pressure.first_reading.systole',
 'health_informations.blood_pressure.second_reading.diastole',
 '_index_map',
 'health_informations.smoking_habit',
 'health_informations.blood_pressure.first_reading.diastole',
 'health_informations.allergies',
 'health_informations.blood_pressure.second_reading.systole',
 '_id',
 'health_informations.blood_type',
 'health_informations.blood_sign']

In [389]:
new_hi_df = hi_df[hi_match_headers]
new_hi_df

NameError: name 'hi_df' is not defined

# PROFILES

In [ ]:
p_df = pd.read_csv( 'data/processed/couchbase-curis-2019-06-21/profiles.csv')
p_df.columns

p_match_headers = list(set(source_keys) & set(p_df.columns))
p_match_headers

In [ ]:
new_p_df = p_df[p_match_headers]
new_p_df

In [ ]:
merge_df = new_p_df.append(new_hi_df, sort=True)
merge_df = merge_df.sort_values(['_id'])
merge_df

In [ ]:
merge_df.to_csv(tmp_dir + 'merge' + '.csv', encoding='utf-8', mode='w', header=True,index=False)